1. Snowpark TRAIN / TEST / PROD access

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col

session = get_active_session()

base_df = session.table("LAB_CALL_CENTER_KM")

train_df = base_df.filter(col("DATASET_SPLIT") == "TRAIN")
test_df  = base_df.filter(col("DATASET_SPLIT") == "TEST")
prod_df  = base_df.filter(col("DATASET_SPLIT") == "PROD")


Quick sanity check:

In [ ]:
for name, df in [("TRAIN", train_df), ("TEST", test_df), ("PROD", prod_df)]:
    print(name, df.count())


2. Feature set for anomaly detection

We’ll keep it numeric only for a clean first pass:

In [ ]:
feature_cols = [
    "HANDLE_TIME_SEC",
    "KM_TIME_SEC",
    "AFTER_CALL_WORK_SEC",
    "HOLD_TIME_SEC",
    "KM_SEARCH_COUNT",
    "KM_ARTICLES_VIEWED",
    "AGENT_TENURE_MONTHS",
    "HOUR_OF_DAY",
    "DAY_OF_WEEK",
]

label_col = "IS_ANOMALY"   # used only for evaluation, not training target
id_col    = "CALL_ID"


Export to pandas to use scikit-learn (make sure scikit-learn and pandas are enabled in your Snowflake Python environment):

In [ ]:
import pandas as pd

train_pd = (
    train_df
    .select([id_col] + feature_cols + [label_col])
    .to_pandas()
)

test_pd = (
    test_df
    .select([id_col] + feature_cols + [label_col])
    .to_pandas()
)

prod_pd = (
    prod_df
    .select([id_col] + feature_cols)
    .to_pandas()
)

X_train = train_pd[feature_cols].values
y_train = train_pd[label_col].values   # for evaluation only
X_test  = test_pd[feature_cols].values
y_test  = test_pd[label_col].values
X_prod  = prod_pd[feature_cols].values


3. Train an Isolation Forest in Snowpark (Python)

We’ll treat this as unsupervised anomaly detection and use the labels only for evaluation.

In [ ]:
from sklearn.ensemble import IsolationForest

# contamination ~ expected anomaly rate (10% from your synthetic design)
iso = IsolationForest(
    n_estimators=200,
    contamination=0.10,
    random_state=42,
)

iso.fit(X_train)


4. Evaluate on TEST (using your IS_ANOMALY labels)

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

# IsolationForest: -1 = anomaly, 1 = normal
test_preds = iso.predict(X_test)
test_scores = iso.decision_function(X_test)  # higher = more normal

# Convert to 0/1 anomaly prediction
test_pred_is_anom = (test_preds == -1).astype(int)

print("Classification report (TEST):")
print(classification_report(y_test, test_pred_is_anom, digits=3))

# AUC using anomaly scores (flip sign so higher score = more anomalous)
test_auc = roc_auc_score(y_test, -test_scores)
print("ROC AUC (TEST):", round(test_auc, 3))


Score the PROD partition and write back to Snowflake

In [ ]:
# Score prod
prod_preds  = iso.predict(X_prod)
prod_scores = iso.decision_function(X_prod)   # higher = more normal

prod_pd["PRED_IS_ANOMALY"] = (prod_preds == -1).astype(int)
prod_pd["ANOMALY_SCORE"]   = -prod_scores    # higher = more anomalous

# Convert back to Snowpark DF
scored_prod_df = session.create_dataframe(prod_pd)

# Persist as a new table
scored_prod_df.write.save_as_table(
    "LAB_CALL_CENTER_KM_PROD_SCORED",
    mode="overwrite"
)


In [ ]:
SELECT CURRENT_DATABASE() AS CURRENT_DB,
       CURRENT_SCHEMA()   AS CURRENT_SCHEMA,
       CURRENT_ROLE()     AS CURRENT_ROLE;


In [ ]:
USE ROLE ACCOUNTADMIN;

USE DATABASE ANOM_DETX;
USE SCHEMA AD_LABS;


In [ ]:
-- In ANOM_DETX.AD_LABS
CREATE OR REPLACE STAGE ML_MODELS_STAGE;


In [ ]:
SHOW STAGES IN SCHEMA ANOM_DETX.AD_LABS;


6. Optional: Persist the model in a stage (for a more production-y pattern)

In [ ]:
import joblib

# Save model locally in the Snowflake Python session
joblib.dump(iso, "/tmp/iso_callcenter_model.joblib")

# Upload to a Snowflake stage
session.file.put(
    "/tmp/iso_callcenter_model.joblib",
    "@ML_MODELS_STAGE",
    overwrite=True
)


In [ ]:
SELECT * FROM ANOM_DETX.AD_LABS.LAB_CALL_CENTER_KM_PROD_SCORED;

7. Where this leaves your “Call Center + KM Anomaly Lab”

You now have:

Synthetic call + KM table (LAB_CALL_CENTER_KM)

Controlled 10% ExtremeLongHandleTime anomalies

TRAIN / TEST / PROD split inside the same table

A Snowpark + scikit-learn anomaly model (IsolationForest)

A scored PROD table usable for:

BI dashboards (top anomalous calls per agent, per call type, per month)

Operational workflows (flag calls for QA review)

Further experimentation (try different features, algorithms, contamination rates)

In [ ]:
CREATE OR REPLACE VIEW VW_CALL_CENTER_ANOMALY_BASE AS
SELECT
    k.CALL_ID,
    k.CALL_START_TS,
    k.CALL_END_TS,
    k.CALL_DATE,
    k.YEAR_MONTH,
    k.DAY_OF_WEEK,
    k.HOUR_OF_DAY,
    k.AGENT_ID,
    k.CUSTOMER_ID,
    k.CALL_TYPE,
    k.CALL_HANDLE_TYPE,
    k.CHANNEL,
    k.AGENT_TENURE_MONTHS,
    k.HANDLE_TIME_SEC,
    k.KM_TIME_SEC,
    k.AFTER_CALL_WORK_SEC,
    k.HOLD_TIME_SEC,
    k.KM_SEARCH_COUNT,
    k.KM_ARTICLES_VIEWED,
    k.IS_RESOLVED,
    k.IS_ESCALATED,
    k.IS_ANOMALY           AS TRUE_LABEL_IS_ANOMALY,
    k.ANOMALY_REASON       AS TRUE_ANOMALY_REASON,
    s.PRED_IS_ANOMALY,
    s.ANOMALY_SCORE
FROM LAB_CALL_CENTER_KM              k
JOIN LAB_CALL_CENTER_KM_PROD_SCORED s
  ON k.CALL_ID = s.CALL_ID
WHERE k.DATASET_SPLIT = 'PROD';


In [ ]:
CREATE OR REPLACE VIEW VW_TOP_ANOMALOUS_CALLS_PER_AGENT_MONTH AS
WITH ranked AS (
    SELECT
        YEAR_MONTH,
        AGENT_ID,
        CALL_ID,
        CALL_DATE,
        CALL_TYPE,
        CHANNEL,
        HANDLE_TIME_SEC,
        KM_TIME_SEC,
        PRED_IS_ANOMALY,
        ANOMALY_SCORE,
        TRUE_LABEL_IS_ANOMALY,
        TRUE_ANOMALY_REASON,
        ROW_NUMBER() OVER (
            PARTITION BY YEAR_MONTH, AGENT_ID
            ORDER BY ANOMALY_SCORE DESC
        ) AS RN
    FROM VW_CALL_CENTER_ANOMALY_BASE
    WHERE PRED_IS_ANOMALY = 1  -- only predicted anomalies
)
SELECT *
FROM ranked
WHERE RN <= 10;  -- “top 10” anomalous calls per agent per month


In [ ]:
CREATE OR REPLACE VIEW VW_TOP_ANOMALOUS_CALLS_PER_CALLTYPE_MONTH AS
WITH ranked AS (
    SELECT
        YEAR_MONTH,
        CALL_TYPE,
        CALL_ID,
        CALL_DATE,
        AGENT_ID,
        CHANNEL,
        HANDLE_TIME_SEC,
        KM_TIME_SEC,
        PRED_IS_ANOMALY,
        ANOMALY_SCORE,
        TRUE_LABEL_IS_ANOMALY,
        TRUE_ANOMALY_REASON,
        ROW_NUMBER() OVER (
            PARTITION BY YEAR_MONTH, CALL_TYPE
            ORDER BY ANOMALY_SCORE DESC
        ) AS RN
    FROM VW_CALL_CENTER_ANOMALY_BASE
    WHERE PRED_IS_ANOMALY = 1
)
SELECT *
FROM ranked
WHERE RN <= 10;


In [ ]:
CREATE OR REPLACE VIEW VW_AGENT_MONTH_ANOMALY_SUMMARY AS
SELECT
    YEAR_MONTH,
    AGENT_ID,
    COUNT(*)                              AS TOTAL_CALLS,
    SUM(PRED_IS_ANOMALY)                  AS PRED_ANOMALY_COUNT,
    AVG(PRED_IS_ANOMALY)                  AS PRED_ANOMALY_RATE,   -- 0–1
    AVG(ANOMALY_SCORE)                    AS AVG_ANOMALY_SCORE,
    MAX(ANOMALY_SCORE)                    AS MAX_ANOMALY_SCORE,
    SUM(CASE WHEN TRUE_LABEL_IS_ANOMALY = 1 THEN 1 ELSE 0 END)
                                          AS TRUE_ANOMALY_COUNT,
    AVG(CASE WHEN TRUE_LABEL_IS_ANOMALY = 1 THEN 1 ELSE 0 END)
                                          AS TRUE_ANOMALY_RATE,
    SUM(CASE WHEN PRED_IS_ANOMALY = TRUE_LABEL_IS_ANOMALY
             THEN 1 ELSE 0 END)           AS MATCH_COUNT
FROM VW_CALL_CENTER_ANOMALY_BASE
GROUP BY YEAR_MONTH, AGENT_ID;


In [ ]:
CREATE OR REPLACE VIEW VW_CALLTYPE_MONTH_ANOMALY_SUMMARY AS
SELECT
    YEAR_MONTH,
    CALL_TYPE,
    COUNT(*)                             AS TOTAL_CALLS,
    SUM(PRED_IS_ANOMALY)                 AS PRED_ANOMALY_COUNT,
    AVG(PRED_IS_ANOMALY)                 AS PRED_ANOMALY_RATE,
    AVG(ANOMALY_SCORE)                   AS AVG_ANOMALY_SCORE,
    MAX(ANOMALY_SCORE)                   AS MAX_ANOMALY_SCORE,
    SUM(CASE WHEN TRUE_LABEL_IS_ANOMALY = 1 THEN 1 ELSE 0 END)
                                         AS TRUE_ANOMALY_COUNT,
    AVG(CASE WHEN TRUE_LABEL_IS_ANOMALY = 1 THEN 1 ELSE 0 END)
                                         AS TRUE_ANOMALY_RATE
FROM VW_CALL_CENTER_ANOMALY_BASE
GROUP BY YEAR_MONTH, CALL_TYPE;
